In [4]:
# ==== 0) Colab setup: core deps ====
!pip -q install kagglehub pandas scikit-learn pillow torch torchvision

In [ ]:
# =========================
# Colab Siamese ISIC-2020 (256x256 mirror)
# =========================

# ---------- 0) Minimal installs ----------
!pip -q install kagglehub tqdm scikit-learn

# ---------- 1) Imports & utilities ----------
from pathlib import Path
import os, re, json, math, random, datetime, numpy as np, pandas as pd
from typing import Optional, Tuple
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import models, transforms as T
from sklearn.model_selection import StratifiedGroupKFold, train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
from tqdm.auto import tqdm

print("PyTorch:", torch.__version__, "| CUDA available:", torch.cuda.is_available())

# ---------- 2) Config (tweak here) ----------
SAVE_TO_DRIVE   = True     # set False if you don't want Drive
USE_IMAGENET    = False    # set True to start from ImageNet weights
IMG_SIZE        = 256      # mirror native size; 384 works too but slower
BATCH_SIZE      = 32       # safe default for Colab T4
EPOCHS          = 8
LR              = 5e-4
WEIGHT_DECAY    = 1e-4
PAIR_RATIO      = (0.6, 0.2, 0.2)   # PN:PP:NN
SAVE_EVERY      = 2                 # save ckpt every N epochs

# Prototype eval knobs (disabled by default for stability/speed)
EMBED_EVERY     = 4      # set to 3 later to enable periodic prototype eval
EMBED_BATCH     = 512
EMBED_MAXITEM   = 4000     # subset for speed (None = all)
EMBED_WORKERS   = 0        # safest on Colab
EMBED_PIN       = False

SEED            = 42
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pin = (device.type == "cuda")
print("Device:", device)

# ---------- 3) Optional: mount Google Drive for persistence ----------
if SAVE_TO_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    RUN_ROOT = Path("/content/drive/MyDrive/siamese_runs")
else:
    RUN_ROOT = Path("/content/siamese_runs")
RUN_DIR = RUN_ROOT / datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
RUN_DIR.mkdir(parents=True, exist_ok=True)
print("Run dir:", RUN_DIR)

# ---------- 4) Robust dataset loader (kagglehub mirror) ----------
import kagglehub

def find_image_dir(root: Path) -> Tuple[Optional[Path], int]:
    exts = {".png",".jpg",".jpeg",".PNG",".JPG",".JPEG"}
    best, best_n = None, 0
    for p in root.rglob("*"):
        if p.is_dir():
            try:
                n = sum(1 for x in p.glob("*") if x.suffix in exts)
            except Exception:
                n = 0
            if n > best_n:
                best, best_n = p, n
    return best, best_n

def pick_csv(root: Path) -> Optional[Path]:
    csvs = list(root.rglob("*.csv"))
    if not csvs: return None
    def score(p: Path):
        try:
            hdr = pd.read_csv(p, nrows=5, low_memory=False)
        except Exception:
            return (-1, p)
        cols = set(hdr.columns)
        label_like = any(c in cols for c in ["target","label","labels","benign_malignant","malignant","is_malignant"])
        id_like    = any(c in cols for c in ["isic_id","image_name","image","image_id","filename","file_name","id","name"])
        return (int(label_like) + int(id_like), p)
    scored = sorted((score(p) for p in csvs), reverse=True)
    return scored[0][1] if scored and scored[0][0] > 0 else csvs[0]

def to_isic_stem(s: str):
    m = re.search(r"(ISIC_\d+)", str(s))
    return m.group(1) if m else None

def index_files_by_stem(img_dir: Path):
    from collections import defaultdict
    by = defaultdict(list)
    for p in img_dir.glob("*"):
        if p.is_file():
            m = re.search(r"(ISIC_\d+)", p.stem)
            if m: by[m.group(1)].append(p)
    return by

ROOT = Path(kagglehub.dataset_download("ziadloo/isic-2020-256x256"))
print("KaggleHub root:", ROOT)

IMG_DIR, n_imgs = find_image_dir(ROOT)
if not IMG_DIR or n_imgs == 0:
    raise SystemExit("Could not locate image files in the mirror.")
CSV = pick_csv(ROOT)
if not CSV:
    raise SystemExit("Could not find a CSV with labels.")
print(f"IMG_DIR: {IMG_DIR}  (files: {n_imgs})")
print(f"CSV: {CSV.name}")

df = pd.read_csv(CSV, low_memory=False)
# id column
cand_id = ["isic_id","image_name","image","image_id","filename","file_name","id","name"]
id_col = next((c for c in cand_id if c in df.columns), None)
if id_col is None:
    for c in df.columns:
        sample = df[c].astype(str).head(50).tolist()
        if sum(bool(re.search(r"ISIC_\d+", s)) for s in sample) >= 10:
            id_col = c; break
if id_col is None:
    raise SystemExit("Could not identify image id column.")

# label column -> target ∈ {0,1}
if "target" in df.columns:
    df["target"] = df["target"].astype(int)
elif "benign_malignant" in df.columns:
    df["target"] = (df["benign_malignant"].astype(str).str.lower()=="malignant").astype(int)
elif "label" in df.columns:
    df["target"] = df["label"].astype(int)
else:
    raise SystemExit("No label column found (target/benign_malignant/label).")

df["image_stem"] = df[id_col].astype(str).map(to_isic_stem)

by_stem = index_files_by_stem(IMG_DIR)
df["image_path"] = df["image_stem"].map(lambda s: str(by_stem[s][0]) if s in by_stem else None)
missing = df["image_path"].isna().sum()
print(f"Mapped images: {len(df)-missing}/{len(df)} (missing={missing})")
df = df.dropna(subset=["image_path"]).reset_index(drop=True)

# Split (patient-level stratified if available)
if "patient_id" in df.columns:
    print("Using patient-level, stratified split.")
    sgkf   = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=SEED)
    groups = df["patient_id"].astype(str)
    y      = df["target"].astype(int)
    tr_idx, va_idx = next(sgkf.split(df, y, groups))
    train_df = df.iloc[tr_idx].reset_index(drop=True)
    val_df   = df.iloc[va_idx].reset_index(drop=True)
else:
    train_df, val_df = train_test_split(
        df, test_size=0.15, stratify=df["target"].astype(int), random_state=SEED
    )
    train_df = train_df.reset_index(drop=True)
    val_df   = val_df.reset_index(drop=True)

print(f"Split sizes → train: {len(train_df)}  val: {len(val_df)}")

# ---------- 5) Torch datasets ----------
mean, std = (0.485,0.456,0.406), (0.229,0.224,0.225)
train_tfms = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.RandomHorizontalFlip(), T.RandomVerticalFlip(),
    T.ToTensor(), T.Normalize(mean, std),
])
eval_tfms = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.ToTensor(), T.Normalize(mean, std),
])

class ISIC2020Dataset(Dataset):
    def __init__(self, frame, transforms=None):
        self.df = frame.reset_index(drop=True)
        self.t  = transforms
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        r = self.df.iloc[i]
        img = Image.open(r["image_path"]).convert("RGB")
        if self.t: img = self.t(img)
        return img, int(r["target"])

ds_train = ISIC2020Dataset(train_df, train_tfms)
ds_val   = ISIC2020Dataset(val_df,   eval_tfms)
print("Datasets ready ✓  train:", len(ds_train), " val:", len(ds_val))

# ---------- 6) Pair dataset & loaders (safe mode: no multiprocessing) ----------
class PairDataset(Dataset):
    """Returns (img1, img2, y_pair) where y_pair=1 if same class (PP/NN), else 0 (PN)."""
    def __init__(self, base_ds, pn_pp_nn=(0.6,0.2,0.2), length=None):
        self.base = base_ds
        self.pn, self.pp, self.nn = pn_pp_nn
        self.length = length or len(base_ds)
        labels = base_ds.df["target"].astype(int).tolist()
        self.pos_idx = [i for i,y in enumerate(labels) if y==1]
        self.neg_idx = [i for i,y in enumerate(labels) if y==0]
        assert self.pos_idx and self.neg_idx, "Both classes required."
    def __len__(self): return self.length
    def __getitem__(self, _):
        r = random.random()
        if r < self.pn:
            i = random.choice(self.pos_idx); j = random.choice(self.neg_idx); y = 0
        elif r < self.pn + self.pp:
            i = random.choice(self.pos_idx); j = random.choice(self.pos_idx); y = 1
        else:
            i = random.choice(self.neg_idx); j = random.choice(self.neg_idx); y = 1
        x1, _ = self.base[i]; x2, _ = self.base[j]
        return x1, x2, y

img_ds_train = ds_train
img_ds_val   = ds_val
pair_ds_train = PairDataset(img_ds_train, pn_pp_nn=PAIR_RATIO, length=len(img_ds_train))
pair_ds_val   = PairDataset(img_ds_val,   pn_pp_nn=(0.5,0.25,0.25), length=len(img_ds_val))

pair_dl_train = DataLoader(pair_ds_train, batch_size=BATCH_SIZE, shuffle=True,
                           num_workers=0, pin_memory=False, persistent_workers=False)
pair_dl_val   = DataLoader(pair_ds_val,   batch_size=BATCH_SIZE, shuffle=False,
                           num_workers=0, pin_memory=False, persistent_workers=False)
print("Pair loaders:", len(pair_dl_train), len(pair_dl_val))

# ---------- 7) Model (Siamese ResNet-18) ----------
class SiameseResNet18(nn.Module):
    def __init__(self, pretrained=False):
        super().__init__()
        m = models.resnet18(weights=None if not pretrained else models.ResNet18_Weights.IMAGENET1K_V1)
        m.fc = nn.Identity()           # 512-d pooled features
        self.encoder = m
        self.scale = nn.Parameter(torch.tensor(10.0))  # temperature for cosine logits
    def forward_once(self, x):
        z = self.encoder(x)
        return F.normalize(z, dim=1)
    def forward(self, x1, x2):
        z1 = self.forward_once(x1); z2 = self.forward_once(x2)
        logits = self.scale * F.cosine_similarity(z1, z2)
        return logits, z1, z2

model = SiameseResNet18(pretrained=USE_IMAGENET).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
criterion = nn.BCEWithLogitsLoss()

# AMP compatibility
def has_torch_amp():
    return hasattr(torch, "amp") and hasattr(torch.amp, "autocast")
def amp_autocast():
    if has_torch_amp():
        return torch.amp.autocast(device_type="cuda", enabled=(device.type=="cuda"))
    else:
        return torch.cuda.amp.autocast(enabled=(device.type=="cuda"))
if has_torch_amp() and hasattr(torch.amp, "GradScaler"):
    scaler = torch.amp.GradScaler(enabled=(device.type=="cuda"))
else:
    scaler = torch.cuda.amp.GradScaler(enabled=(device.type=="cuda"))

# ---------- 8) Prototype evaluation (optional; off by default) ----------
def _maybe_subset(ds, max_items):
    if (max_items is None) or (len(ds) <= max_items): return ds
    idx = np.random.choice(len(ds), size=max_items, replace=False)
    return Subset(ds, idx.tolist())

@torch.no_grad()
def compute_embeddings(ds, batch=EMBED_BATCH, max_items=EMBED_MAXITEM):
    ds_sub = _maybe_subset(ds, max_items)
    dl = DataLoader(ds_sub, batch_size=batch, shuffle=False,
                    num_workers=EMBED_WORKERS, pin_memory=EMBED_PIN,
                    persistent_workers=False)
    all_z, all_y = [], []
    model.eval()
    for xb, yb in tqdm(dl, desc="Embed", leave=False):
        xb = xb.to(device, non_blocking=False)
        with amp_autocast():
            z = model.forward_once(xb)
        all_z.append(z.cpu()); all_y.append(yb)
    return torch.cat(all_z, 0), torch.cat(all_y, 0)

@torch.no_grad()
def prototype_eval(train_ds, val_ds):
    z_tr, y_tr = compute_embeddings(train_ds)
    c0 = F.normalize(z_tr[y_tr==0].mean(0, keepdim=True), dim=1) if (y_tr==0).any() else None
    c1 = F.normalize(z_tr[y_tr==1].mean(0, keepdim=True), dim=1) if (y_tr==1).any() else None
    z_va, y_va = compute_embeddings(val_ds)
    s1 = F.cosine_similarity(z_va, c1.expand_as(z_va)) if c1 is not None else torch.zeros(len(z_va))
    s0 = F.cosine_similarity(z_va, c0.expand_as(z_va)) if c0 is not None else torch.zeros(len(z_va))
    scores = (s1 - s0).cpu().numpy()
    y_true = y_va.numpy()
    y_pred = (scores > 0).astype(int)
    acc = accuracy_score(y_true, y_pred)
    try:
        auc = roc_auc_score(y_true, scores)
    except ValueError:
        auc = float("nan")
    return acc, auc

# ---------- 9) (Optional) resume from latest ckpt in RUN_ROOT ----------
def find_latest_ckpt(root):
    pts = sorted(Path(root).rglob("siamese_resnet18_epoch*.pt"))
    return pts[-1] if pts else None

latest = find_latest_ckpt(RUN_ROOT)
start_epoch = 1
if latest:
    try:
        ck = torch.load(latest, map_location="cpu")
        model.load_state_dict(ck["model_state"])
        optimizer.load_state_dict(ck["optimizer_state"])
        start_epoch = int(ck.get("epoch", 0)) + 1
        print(f"Resumed from {latest} @ epoch {start_epoch}")
    except Exception as e:
        print("Resume failed:", e)

# ---------- 10) Train ----------
metrics_path = RUN_DIR / "metrics.csv"
with open(metrics_path, "w") as f:
    f.write("epoch,train_loss,val_pair_loss,val_proto_acc,val_proto_auc\n")

best_acc = -1.0

for epoch in range(start_epoch, EPOCHS+1):
    # Train
    model.train()
    running, n_batches = 0.0, 0
    pbar = tqdm(pair_dl_train, desc=f"Epoch {epoch}/{EPOCHS} [train]", leave=False)
    for x1, x2, y in pbar:
        x1 = x1.to(device, non_blocking=True)
        x2 = x2.to(device, non_blocking=True)
        y  = y.float().to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        with amp_autocast():
            logits, _, _ = model(x1, x2)
            loss = criterion(logits, y)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running += loss.item(); n_batches += 1
        pbar.set_postfix(loss=f"{loss.item():.4f}")

    train_loss = running / max(1, n_batches)

    # Val (pair loss)
    model.eval()
    val_running, val_nb = 0.0, 0
    pbar_v = tqdm(pair_dl_val, desc=f"Epoch {epoch}/{EPOCHS} [val pairs]", leave=False)
    with torch.no_grad():
        for x1, x2, y in pbar_v:
            x1 = x1.to(device, non_blocking=True)
            x2 = x2.to(device, non_blocking=True)
            y  = y.float().to(device, non_blocking=True)
            with amp_autocast():
                logits, _, _ = model(x1, x2)
                vloss = criterion(logits, y)
            val_running += vloss.item(); val_nb += 1
            pbar_v.set_postfix(loss=f"{vloss.item():.4f}")
    val_pair_loss = val_running / max(1, val_nb)

    # Prototype eval (disabled unless EMBED_EVERY divides epoch)
    if EMBED_EVERY > 0 and (epoch % EMBED_EVERY == 0):
        val_proto_acc, val_proto_auc = prototype_eval(img_ds_train, img_ds_val)
    else:
        val_proto_acc, val_proto_auc = float("nan"), float("nan")

    # Log
    with open(metrics_path, "a") as f:
        f.write(f"{epoch},{train_loss:.6f},{val_pair_loss:.6f},{val_proto_acc:.6f},{val_proto_auc:.6f}\n")

    print(f"[{epoch:02d}/{EPOCHS}] train_loss={train_loss:.4f} | "
          f"val_pair_loss={val_pair_loss:.4f} | proto_acc={val_proto_acc:.4f} | proto_auc={val_proto_auc:.4f}")

    # Save every N epochs
    if epoch % SAVE_EVERY == 0:
        ckpt_path = RUN_DIR / f"siamese_resnet18_epoch{epoch:02d}.pt"
        torch.save({
            "epoch": epoch,
            "model_state": model.state_dict(),
            "optimizer_state": optimizer.state_dict(),
            "config": {
                "lr": LR, "batch_size": BATCH_SIZE, "pair_ratio": PAIR_RATIO,
                "seed": SEED, "img_size": IMG_SIZE, "embed_every": EMBED_EVERY
            }
        }, ckpt_path)
        print("Saved:", ckpt_path)

    # Track best (when proto eval runs)
    if not math.isnan(val_proto_acc) and (val_proto_acc > best_acc):
        best_acc = val_proto_acc
        torch.save({"epoch": epoch, "model_state": model.state_dict()}, RUN_DIR / "best.pt")

# Save run config
with open(RUN_DIR / "config.json", "w") as f:
    json.dump({
        "lr": LR, "batch_size": BATCH_SIZE, "epochs": EPOCHS, "pair_ratio": PAIR_RATIO,
        "seed": SEED, "device": str(device), "img_size": IMG_SIZE,
        "embed_every": EMBED_EVERY, "embed_batch": EMBED_BATCH, "embed_maxitem": EMBED_MAXITEM,
        "save_every": SAVE_EVERY, "use_imagenet": USE_IMAGENET, "save_to_drive": SAVE_TO_DRIVE
    }, f, indent=2)

print("Done ✓")
print("Run dir:", RUN_DIR)
print("Metrics:", metrics_path)


PyTorch: 2.8.0+cu126 | CUDA available: True
Device: cuda
Mounted at /content/drive
Run dir: /content/drive/MyDrive/siamese_runs/20251028_150931
Using Colab cache for faster access to the 'isic-2020-256x256' dataset.
KaggleHub root: /kaggle/input/isic-2020-256x256
IMG_DIR: /kaggle/input/isic-2020-256x256/train-image/image  (files: 33126)
CSV: train-metadata.csv
Mapped images: 33126/33126 (missing=0)
Using patient-level, stratified split.
Split sizes → train: 26466  val: 6660
Datasets ready ✓  train: 26466  val: 6660
Pair loaders: 828 209
Resumed from /content/drive/MyDrive/siamese_runs/20251028_144116/siamese_resnet18_epoch04.pt @ epoch 5


Epoch 5/8 [train]:   0%|          | 0/828 [00:00<?, ?it/s]

In [11]:
# === Post-training evaluation (same session) ===
import numpy as np, torch, torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score, average_precision_score, confusion_matrix
from tqdm.auto import tqdm
from torchvision import transforms as T
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EMBED_BATCH   = 512
EMBED_WORKERS = 0
EMBED_PIN     = False

# 1) Make eval-only dataset clones (no augmentations)
eval_tfms = T.Compose([T.Resize((IMG_SIZE, IMG_SIZE)),
                       T.ToTensor(),
                       T.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))])

class ISIC2020Dataset(torch.utils.data.Dataset):
    def __init__(self, frame, transforms=None):
        self.df = frame.reset_index(drop=True)
        self.t  = transforms
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        r = self.df.iloc[i]
        img = Image.open(r["image_path"]).convert("RGB")
        if self.t: img = self.t(img)
        return img, int(r["target"])

# reuse frame from the loader cell if available
ds_tr_eval = ISIC2020Dataset(train_df, eval_tfms) if 'train_df' in globals() else ds_train
ds_va_eval = ISIC2020Dataset(val_df,   eval_tfms) if 'val_df'   in globals() else ds_val

# 2) Helpers
@torch.no_grad()
def compute_embeddings(model, ds, batch=EMBED_BATCH):
    dl = DataLoader(ds, batch_size=batch, shuffle=False,
                    num_workers=EMBED_WORKERS, pin_memory=EMBED_PIN)
    all_z, all_y = [], []
    model.eval()
    for xb, yb in tqdm(dl, desc="Embed", leave=False):
        xb = xb.to(device, non_blocking=False)
        with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
            z = model.forward_once(xb)
        all_z.append(z.cpu()); all_y.append(yb)
    return torch.cat(all_z, 0), torch.cat(all_y, 0)

@torch.no_grad()
def prototype_metrics(model, train_ds, val_ds):
    z_tr, y_tr = compute_embeddings(model, train_ds)
    c0 = F.normalize(z_tr[y_tr==0].mean(0, keepdim=True), dim=1) if (y_tr==0).any() else None
    c1 = F.normalize(z_tr[y_tr==1].mean(0, keepdim=True), dim=1) if (y_tr==1).any() else None
    z_va, y_va = compute_embeddings(model, val_ds)
    s1 = F.cosine_similarity(z_va, c1.expand_as(z_va)) if c1 is not None else torch.zeros(len(z_va))
    s0 = F.cosine_similarity(z_va, c0.expand_as(z_va)) if c0 is not None else torch.zeros(len(z_va))
    scores = (s1 - s0).cpu().numpy()
    y_true = y_va.numpy()
    y_pred = (scores > 0).astype(int)

    acc      = accuracy_score(y_true, y_pred)
    bal_acc  = balanced_accuracy_score(y_true, y_pred)
    try:
        roc = roc_auc_score(y_true, scores)
        pr  = average_precision_score(y_true, scores)  # PR-AUC
    except ValueError:
        roc, pr = float("nan"), float("nan")
    cm = confusion_matrix(y_true, y_pred, labels=[0,1])
    return acc, bal_acc, roc, pr, cm

def make_balanced_subset(ds):
    # Balanced subset for human-friendly accuracy (doesn't change training distribution)
    if not hasattr(ds, "df"): return ds  # fallback
    y = ds.df["target"].to_numpy()
    pos = np.where(y==1)[0]; neg = np.where(y==0)[0]
    if len(pos)==0 or len(neg)==0: return ds
    n = min(len(pos), len(neg))
    idx = np.concatenate([np.random.choice(pos, n, replace=False),
                          np.random.choice(neg, n, replace=False)])
    return Subset(ds, idx.tolist())

# 3) Run metrics: imbalanced val + balanced val subset
print("== Imbalanced validation (real prevalence) ==")
acc, bal_acc, roc, pr, cm = prototype_metrics(model, ds_tr_eval, ds_va_eval)
print(f"ACC={acc:.4f}  BAL_ACC={bal_acc:.4f}  ROC-AUC={roc:.4f}  PR-AUC={pr:.4f}\nCM=\n{cm}")

print("\n== Balanced validation subset (1:1 sanity check) ==")
va_bal = make_balanced_subset(ds_va_eval)
acc, bal_acc, roc, pr, cm = prototype_metrics(model, ds_tr_eval, va_bal)
print(f"ACC={acc:.4f}  BAL_ACC={bal_acc:.4f}  ROC-AUC={roc:.4f}  PR-AUC={pr:.4f}\nCM=\n{cm}")


== Imbalanced validation (real prevalence) ==


Embed:   0%|          | 0/52 [00:00<?, ?it/s]

/tmp/ipython-input-2756004303.py:43: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):


Embed:   0%|          | 0/14 [00:00<?, ?it/s]

ACC=0.4971  BAL_ACC=0.6876  ROC-AUC=0.7106  PR-AUC=0.0370
CM=
[[3203 3335]
 [  14  108]]

== Balanced validation subset (1:1 sanity check) ==


Embed:   0%|          | 0/52 [00:00<?, ?it/s]

/tmp/ipython-input-2756004303.py:43: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):


Embed:   0%|          | 0/1 [00:00<?, ?it/s]

ACC=0.6762  BAL_ACC=0.6762  ROC-AUC=0.7158  PR-AUC=0.6723
CM=
[[ 57  65]
 [ 14 108]]
